# Combine Data.ipynb

Merge all datasets together before conducting analysis.

Datasets to combine: Automated_Grading.xlsx, Automated_Grading_Experiments.xlsx, Automatic_Metrics.xlsx

Add useful variables

Additionally create a dataset for comparison with direct prompting

In [1]:
import pandas as pd


In [5]:
# Concatenate cw_graded.xlsx and gsm8k_graded.xlsx
cw_grading = pd.read_excel('cw_graded.xlsx')
gsm8k_grading = pd.read_excel('gsm8k_graded.xlsx')

stacked_df = pd.concat([cw_grading, gsm8k_grading])
stacked_df


,model_task_method,conversation_number,coherence_1_incoherent_10_very_coherent,task_constraints_followed_0_not_followed_1_followed,ease_of_review_1_easy_10_hard,correct
0,td3_cw_direct_prompting_responses,1,1.0,1.0,NaN,NaN
1,td3_cw_direct_prompting_responses,2,7.0,0.0,NaN,NaN
2,td3_cw_direct_prompting_responses,3,1.0,1.0,NaN,NaN
3,td3_cw_direct_prompting_responses,4,10.0,1.0,NaN,NaN
4,td3_cw_direct_prompting_responses,5,4.0,1.0,NaN,NaN
...,...,...,...,...,...,...
1595,gpt4_gsm8k_manual_cot_responses,96,NaN,NaN,NaN,1.0
1596,gpt4_gsm8k_manual_cot_responses,97,NaN,NaN,NaN,1.0
1597,gpt4_gsm8k_manual_cot_responses,98,NaN,NaN,NaN,1.0
1598,gpt4_gsm8k_manual_cot_responses,99,NaN,NaN,NaN,1.0


### Adding fine-tuned GPT-3.5 predictions

In [ ]:
# Predictions from fine-tuned GPT-3.5
# Merge on "predictions_based_on_first_10.csv", "items_to_predict_using_last_10.csv"
# Limit columns to model_task_method, conversation_number, Prediction

first_10 = pd.read_csv('predictions_based_on_first_10.csv')[['model_task_method', 'conversation_number', 'Prediction']]
# Rename column Prediction to Prediction_Based_On_First_10
first_10.rename(columns={'Prediction': 'Prediction_Based_On_First_10'}, inplace=True)

last_10 = pd.read_csv('items_to_predict_using_last_10.csv')[['model_task_method', 'conversation_number', 'Prediction']]
# Rename column Prediction to Prediction_Based_On_Last_10
last_10.rename(columns={'Prediction': 'Prediction_Based_On_Last_10'}, inplace=True)

print(first_10)
print(last_10)


In [ ]:
# Merge on model_task_method and conversation_number
stacked_df_w_pred = stacked_df.merge(first_10, on=['model_task_method', 'conversation_number'], how = 'left')
stacked_df_w_pred = stacked_df_w_pred.merge(last_10, on=['model_task_method', 'conversation_number'], how = 'left')


In [ ]:
# Remove {"Score": "
# Remove "}
stacked_df_w_pred['Prediction_Based_On_First_10'] = stacked_df_w_pred['Prediction_Based_On_First_10'].str.replace('{"Score": "', '')
stacked_df_w_pred['Prediction_Based_On_First_10'] = stacked_df_w_pred['Prediction_Based_On_First_10'].str.replace('"}', '')
stacked_df_w_pred['Prediction_Based_On_Last_10'] = stacked_df_w_pred['Prediction_Based_On_Last_10'].str.replace('{"Score": "', '')
stacked_df_w_pred['Prediction_Based_On_Last_10'] = stacked_df_w_pred['Prediction_Based_On_Last_10'].str.replace('"}', '')

# Convert score to float
stacked_df_w_pred['Prediction_Based_On_First_10'] = stacked_df_w_pred['Prediction_Based_On_First_10'].astype(float)
stacked_df_w_pred['Prediction_Based_On_Last_10'] = stacked_df_w_pred['Prediction_Based_On_Last_10'].astype(float)

# Print some values
print(stacked_df_w_pred['Prediction_Based_On_First_10'].head())


In [ ]:
# Create column "Aggegated_Prediction" as the mean of the two predictions
# If one is missing, use the other
stacked_df_w_pred['Aggegated_Prediction'] = stacked_df_w_pred[['Prediction_Based_On_First_10', 'Prediction_Based_On_Last_10']].mean(axis=1)
stacked_df_w_pred['Aggegated_Prediction'] = stacked_df_w_pred['Aggegated_Prediction'].fillna(stacked_df_w_pred['Prediction_Based_On_First_10'])
stacked_df_w_pred['Aggegated_Prediction'] = stacked_df_w_pred['Aggegated_Prediction'].fillna(stacked_df_w_pred['Prediction_Based_On_Last_10'])


In [6]:
# Automatic_Metrics.xlsx
Automatic_Metrics = pd.read_excel('Automatic_Metrics.xlsx')

Automatic_Metrics


,model_task_method,conversation_number,conversation_length,input_length,output_length,conversation_cost,gsm8k_question_index,gsm8k_answer,gsm8k_length_vs_provided,length_vs_direct_prompting,...,num_linebreaks_prompts,num_sentences_prompts,num_step_i_prompts,num_1_dot_etc_prompts,sentence_length_prompts,fres_prompts,num_linebreaks_provided,num_sentences_provided,num_step_i_provided,num_1_dot_etc_provided
0,td3_gsm8k_direct_prompting_responses,1,82,69,12,0.00164,0.0,Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eg...,0.689076,1.000000,...,1,5,0,0,NaN,NaN,2.0,3.0,0.0,0.0
1,td3_gsm8k_direct_prompting_responses,2,36,30,5,0.00072,1.0,It takes 2/2=<<2/2=1>>1 bolt of white fiber\nS...,0.514286,1.000000,...,1,3,0,0,NaN,NaN,2.0,1.0,0.0,0.0
2,td3_gsm8k_direct_prompting_responses,3,104,53,50,0.00208,2.0,The cost of the house and repairs came out to ...,0.608187,1.000000,...,1,5,0,0,NaN,NaN,4.0,1.0,0.0,2.0
3,td3_gsm8k_direct_prompting_responses,4,39,36,2,0.00078,3.0,He sprints 3*3=<<3*3=9>>9 times\nSo he runs 9*...,0.573529,1.000000,...,1,4,0,0,NaN,NaN,2.0,1.0,0.0,0.0
4,td3_gsm8k_direct_prompting_responses,5,115,109,5,0.00230,4.0,"If each chicken eats 3 cups of feed per day, t...",0.598958,1.000000,...,1,6,0,0,NaN,NaN,2.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,gpt4_cw_manual_cot_responses,96,949,704,245,0.03582,NaN,NaN,NaN,2.711429,...,0,35,0,12,17.971429,73.78,NaN,NaN,NaN,NaN
3196,gpt4_cw_manual_cot_responses,97,1012,714,298,0.03930,NaN,NaN,NaN,2.899713,...,0,35,0,12,18.228571,73.58,NaN,NaN,NaN,NaN
3197,gpt4_cw_manual_cot_responses,98,979,711,268,0.03741,NaN,NaN,NaN,2.734637,...,0,35,0,12,18.200000,73.58,NaN,NaN,NaN,NaN
3198,gpt4_cw_manual_cot_responses,99,1023,703,320,0.04029,NaN,NaN,NaN,3.177019,...,0,35,0,12,17.971429,73.78,NaN,NaN,NaN,NaN


In [7]:
# Merge on model_task_method, conversation_number
merged_df = pd.merge(stacked_df_w_pred, Automatic_Metrics, on=['model_task_method', 'conversation_number'], how='left')

merged_df


,model_task_method,conversation_number,coherence_1_incoherent_10_very_coherent,task_constraints_followed_0_not_followed_1_followed,ease_of_review_1_easy_10_hard,correct,conversation_length,input_length,output_length,conversation_cost,...,num_linebreaks_prompts,num_sentences_prompts,num_step_i_prompts,num_1_dot_etc_prompts,sentence_length_prompts,fres_prompts,num_linebreaks_provided,num_sentences_provided,num_step_i_provided,num_1_dot_etc_provided
0,td3_cw_direct_prompting_responses,1,1.0,1.0,NaN,NaN,177,52,124,0.00354,...,0,5,0,2,10.0,94.35,NaN,NaN,NaN,NaN
1,td3_cw_direct_prompting_responses,2,7.0,0.0,NaN,NaN,240,62,177,0.00480,...,0,5,0,2,11.4,86.20,NaN,NaN,NaN,NaN
2,td3_cw_direct_prompting_responses,3,1.0,1.0,NaN,NaN,241,68,172,0.00482,...,0,5,0,2,12.2,74.69,NaN,NaN,NaN,NaN
3,td3_cw_direct_prompting_responses,4,10.0,1.0,NaN,NaN,174,48,125,0.00348,...,0,5,0,2,9.0,69.99,NaN,NaN,NaN,NaN
4,td3_cw_direct_prompting_responses,5,4.0,1.0,NaN,NaN,239,68,170,0.00478,...,0,5,0,2,12.0,74.90,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,gpt4_gsm8k_manual_cot_responses,96,NaN,NaN,NaN,1.0,833,725,108,0.02823,...,0,45,0,20,NaN,NaN,4.0,5.0,0.0,1.0
3196,gpt4_gsm8k_manual_cot_responses,97,NaN,NaN,NaN,1.0,778,717,61,0.02517,...,0,46,0,20,NaN,NaN,2.0,3.0,0.0,0.0
3197,gpt4_gsm8k_manual_cot_responses,98,NaN,NaN,NaN,1.0,847,737,110,0.02871,...,0,47,0,20,NaN,NaN,3.0,4.0,0.0,0.0
3198,gpt4_gsm8k_manual_cot_responses,99,NaN,NaN,NaN,1.0,875,776,99,0.02922,...,0,47,0,20,NaN,NaN,2.0,3.0,0.0,0.0


### CW Compliance

In [8]:
# Add Creative_Writing_Compliance.xlsx
Creative_Writing_Compliance = pd.read_excel('Creative_Writing_Compliance.xlsx')

merged_df = pd.merge(merged_df, Creative_Writing_Compliance, on=['model_task_method', 'conversation_number'], how='left')
merged_df


,model_task_method,conversation_number,coherence_1_incoherent_10_very_coherent,task_constraints_followed_0_not_followed_1_followed,ease_of_review_1_easy_10_hard,correct,conversation_length,input_length,output_length,conversation_cost,...,num_sentences_prompts,num_step_i_prompts,num_1_dot_etc_prompts,sentence_length_prompts,fres_prompts,num_linebreaks_provided,num_sentences_provided,num_step_i_provided,num_1_dot_etc_provided,compliance
0,td3_cw_direct_prompting_responses,1,1.0,1.0,NaN,NaN,177,52,124,0.00354,...,5,0,2,10.0,94.35,NaN,NaN,NaN,NaN,1.0
1,td3_cw_direct_prompting_responses,2,7.0,0.0,NaN,NaN,240,62,177,0.00480,...,5,0,2,11.4,86.20,NaN,NaN,NaN,NaN,0.0
2,td3_cw_direct_prompting_responses,3,1.0,1.0,NaN,NaN,241,68,172,0.00482,...,5,0,2,12.2,74.69,NaN,NaN,NaN,NaN,1.0
3,td3_cw_direct_prompting_responses,4,10.0,1.0,NaN,NaN,174,48,125,0.00348,...,5,0,2,9.0,69.99,NaN,NaN,NaN,NaN,1.0
4,td3_cw_direct_prompting_responses,5,4.0,1.0,NaN,NaN,239,68,170,0.00478,...,5,0,2,12.0,74.90,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,gpt4_gsm8k_manual_cot_responses,96,NaN,NaN,NaN,1.0,833,725,108,0.02823,...,45,0,20,NaN,NaN,4.0,5.0,0.0,1.0,NaN
3196,gpt4_gsm8k_manual_cot_responses,97,NaN,NaN,NaN,1.0,778,717,61,0.02517,...,46,0,20,NaN,NaN,2.0,3.0,0.0,0.0,NaN
3197,gpt4_gsm8k_manual_cot_responses,98,NaN,NaN,NaN,1.0,847,737,110,0.02871,...,47,0,20,NaN,NaN,3.0,4.0,0.0,0.0,NaN
3198,gpt4_gsm8k_manual_cot_responses,99,NaN,NaN,NaN,1.0,875,776,99,0.02922,...,47,0,20,NaN,NaN,2.0,3.0,0.0,0.0,NaN


In [ ]:
# Create coherence_1_incoherent_10_very_coherent_compliance_adjusted
# This is coherence_1_incoherent_10_very_coherent but if the conversation is not compliant, the score is 1
merged_df['coherence_1_incoherent_10_very_coherent_compliance_adjusted'] = merged_df['coherence_1_incoherent_10_very_coherent']
merged_df.loc[merged_df['compliance'] == 0, 'coherence_1_incoherent_10_very_coherent_compliance_adjusted'] = 1


## Adding Columns

In [9]:
# Get model, task, and method from model_task_method
# Model - first item when split on underscores
# Task - second item when split on underscores
# method - all remaining items when split on underscores, removing the last item
merged_df['model'] = merged_df['model_task_method'].str.split('_').str[0]
merged_df['task'] = merged_df['model_task_method'].str.split('_').str[1]
merged_df['method'] = merged_df['model_task_method'].str.split('_').str[2:].str[:-1].str.join('_')

# Print value counts of model, task, and method
print(merged_df['model'].value_counts())
print(merged_df['task'].value_counts())
print(merged_df['method'].value_counts())


model
td3     1600
gpt4    1600
Name: count, dtype: int64
task
cw       1600
gsm8k    1600
Name: count, dtype: int64
method
direct_prompting     400
zero_shot_cot        400
ape_zero_shot_cot    400
least_to_most        400
manual_few_shot      400
manual_cot           400
tree_of_thought      400
self_refine          400
Name: count, dtype: int64


In [10]:
# Column for combined accuracy_quality - correct_or_incorrect if task = "gsm8k", creative_writing_score if task = "cw"
merged_df['accuracy_quality'] = merged_df.apply(lambda row: row['correct'] if row['task'] == 'gsm8k' else row['coherence_1_incoherent_10_very_coherent'], axis=1)

# Also create accuracy_quality_compliance_adjusted
merged_df['accuracy_quality_compliance_adjusted'] = merged_df.apply(lambda row: row['correct'] if row['task'] == 'gsm8k' else row['coherence_1_incoherent_10_very_coherent_compliance_adjusted'], axis=1)


## Combined Output

In [11]:
# Output Combined_Data.xlsx
merged_df.to_excel('Combined_Data.xlsx', index=False)


## Direct Prompting Comparison

In [12]:
# Split dataset
combined_data = pd.read_excel('Combined_Data.xlsx')
no_direct_prompting_data = combined_data[combined_data['method'] != 'direct_prompting']
direct_prompting_data = combined_data[combined_data['method'] == 'direct_prompting']

# Add prefix dp_ to columns in direct_prompting_data
direct_prompting_data = direct_prompting_data.add_prefix('dp_')

# Left join datasets on model = dp_model, task = dp_task
direct_prompting_comparison = no_direct_prompting_data.merge(direct_prompting_data, left_on=['model', 'task', 'conversation_number'], right_on=['dp_model', 'dp_task', 'dp_conversation_number'], how='left')

direct_prompting_comparison


,model_task_method,conversation_number,coherence_1_incoherent_10_very_coherent,task_constraints_followed_0_not_followed_1_followed,ease_of_review_1_easy_10_hard,correct,conversation_length,input_length,output_length,conversation_cost,...,dp_fres_prompts,dp_num_linebreaks_provided,dp_num_sentences_provided,dp_num_step_i_provided,dp_num_1_dot_etc_provided,dp_compliance,dp_model,dp_task,dp_method,dp_accuracy_quality
0,td3_cw_zero_shot_cot_responses,1,2.0,1.0,2.0,NaN,278,63,214,0.00556,...,94.35,NaN,NaN,NaN,NaN,1.0,td3,cw,direct_prompting,1
1,td3_cw_zero_shot_cot_responses,2,8.0,1.0,3.0,NaN,264,73,190,0.00528,...,86.20,NaN,NaN,NaN,NaN,0.0,td3,cw,direct_prompting,7
2,td3_cw_zero_shot_cot_responses,3,8.0,1.0,2.0,NaN,295,79,215,0.00590,...,74.69,NaN,NaN,NaN,NaN,1.0,td3,cw,direct_prompting,1
3,td3_cw_zero_shot_cot_responses,4,9.0,0.0,1.0,NaN,258,59,198,0.00516,...,69.99,NaN,NaN,NaN,NaN,1.0,td3,cw,direct_prompting,10
4,td3_cw_zero_shot_cot_responses,5,1.0,1.0,6.0,NaN,322,79,242,0.00644,...,74.90,NaN,NaN,NaN,NaN,1.0,td3,cw,direct_prompting,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2795,gpt4_gsm8k_manual_cot_responses,96,NaN,NaN,NaN,1.0,833,725,108,0.02823,...,NaN,4.0,5.0,0.0,1.0,NaN,gpt4,gsm8k,direct_prompting,1
2796,gpt4_gsm8k_manual_cot_responses,97,NaN,NaN,NaN,1.0,778,717,61,0.02517,...,NaN,2.0,3.0,0.0,0.0,NaN,gpt4,gsm8k,direct_prompting,0
2797,gpt4_gsm8k_manual_cot_responses,98,NaN,NaN,NaN,1.0,847,737,110,0.02871,...,NaN,3.0,4.0,0.0,0.0,NaN,gpt4,gsm8k,direct_prompting,0
2798,gpt4_gsm8k_manual_cot_responses,99,NaN,NaN,NaN,1.0,875,776,99,0.02922,...,NaN,2.0,3.0,0.0,0.0,NaN,gpt4,gsm8k,direct_prompting,1


In [13]:
# Save direct_prompting_comparison to Excel
direct_prompting_comparison.to_excel('direct_prompting_comparison.xlsx', index=False)
